# SSEBop Single Image Example

In [1]:
import ee
from IPython.display import Image
from importlib import reload  # Python 3 only
# from imp import reload      # Python 2/3 via futures modules

from ssebop import ssebop
ssebop = reload(ssebop)

ee.Initialize()

ndvi_palette = ['#EFE7E1', '#003300']
# ta_palette = ['#ff0000', '#ffff00', '#00ffff', '#0000ff']
et_palette = [
  'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
  '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']

# Input Landsat 8 Image

In [2]:
landsat_img = ee.Image('LANDSAT/LC08/C01/T1_RT_TOA/LC08_043033_20150805')
Image(url=landsat_img.select([3, 2, 1]).getThumbURL({'min': 0.0, 'max': 0.3}))

In [3]:
# Build the SSEBop object
ssebop_obj = ssebop.SSEBop.fromLandsatTOA(landsat_img)

In [4]:
# Get NDVI from the object
Image(url=ssebop_obj.ndvi.getThumbURL(
    {'min': -0.1, 'max': 0.9, 'palette': ','.join(ndvi_palette)}))

In [5]:
# Compute ETf for a single image
Image(url=ssebop_obj.etf() \
    .getThumbURL({'min': 0.0, 'max': 1.2, 'palette': ','.join(et_palette)}))

# Build from component images

In [6]:
# For this example, use brightness temperature directly instead of computing LST
input_image = ee.Image([
        landsat_img.normalizedDifference(['B5', 'B4']).rename(['ndvi']),
        landsat_img.select(['B10'], ['lst'])
    ]).setMulti({
        'system:index': landsat_img.get('system:index'),
        'system:time_start': landsat_img.get('system:time_start')})
Image(url=ssebop.SSEBop(input_image).etf() \
    .getThumbURL({'min': 0.0, 'max': 1.2, 'palette': ','.join(et_palette)}))